<a href="https://www.kaggle.com/code/kuntalpal/pycaret-automl-regression?scriptVersionId=126516855" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wids-2023/train_data.csv
/kaggle/input/wids-2023/test_data.csv
/kaggle/input/wids-2023/sample_solution.csv


In [2]:
import numpy as np
import pandas as pd
import IPython
import IPython.display
import tensorflow as tf
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.utils import shuffle

In [3]:
train_df = pd.read_csv('/kaggle/input/wids-2023/train_data.csv')
test_df = pd.read_csv("/kaggle/input/wids-2023/test_data.csv")
sample = pd.read_csv('/kaggle/input/wids-2023/sample_solution.csv')

print(f"train_shape:{train_df.shape},test_shape:{test_df.shape},Sample_shape:{sample.shape}")

train_shape:(375734, 246),test_shape:(31354, 245),Sample_shape:(31354, 2)


In [4]:
# Explore how the datatypes look like for all the columns
count=0
for feature in list(train_df.columns):
    
    if train_df[feature].dtype == 'float64':
        count +=1
    else:
        print(feature,train_df[feature].dtype)
print(count)
        

index int64
startdate object
climateregions__climateregion object
mjo1d__phase int64
mei__meirank int64
mei__nip int64
240


In [5]:
# Check out for missing values in training data
all_features_nan =  train_df.columns[train_df.isna().any()].tolist()


for feature in all_features_nan:
    print(feature,train_df[feature].isna().sum())

nmme0-tmp2m-34w__ccsm30 15934
nmme-tmp2m-56w__ccsm3 10280
nmme-prate-34w__ccsm3 8738
nmme0-prate-56w__ccsm30 15934
nmme0-prate-34w__ccsm30 15934
nmme-prate-56w__ccsm3 10280
nmme-tmp2m-34w__ccsm3 8738
ccsm30 15934


In [6]:
# Range of dates in training and test dataset
train_df['startdate'] = pd.to_datetime(train_df['startdate'])
test_df['startdate'] = pd.to_datetime(test_df['startdate'])
print(train_df['startdate'].min(),train_df['startdate'].max())
print(test_df['startdate'].min(),test_df['startdate'].max())

2014-09-01 00:00:00 2016-08-31 00:00:00
2022-11-01 00:00:00 2022-12-31 00:00:00


In [7]:
# Group places together by latitude and longtitude
train_df.groupby(['lat','lon']).size()

lat       lon     
0.000000  0.833333    731
          0.866667    731
0.045455  0.833333    731
          0.866667    731
          0.900000    731
                     ... 
1.000000  0.733333    731
          0.766667    731
          0.800000    731
          0.833333    731
          0.866667    731
Length: 514, dtype: int64

In [8]:
test_df.groupby(['lat','lon']).ngroup()

0          0
1          0
2          0
3          0
4          0
        ... 
31349    513
31350    513
31351    513
31352    513
31353    513
Length: 31354, dtype: int64

In [9]:
# Imputing Missing values
imputer = SimpleImputer(strategy='mean')
for feature in all_features_nan:
    train_df[feature] = imputer.fit_transform(train_df[[feature]]).ravel()

In [10]:
# Round off the latitude and longitude till the 14th place
train_df.loc[:,'lat']=round(train_df.lat,14)
train_df.loc[:,'lon']=round(train_df.lon,14)
test_df.loc[:,'lat']=round(test_df.lat,14)
test_df.loc[:,'lon']=round(test_df.lon,14)

In [11]:
# transform non-numerical labels  to numerical labels.
le = preprocessing.LabelEncoder()
train_df['climateregions__climateregion'] = le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion'] = le.transform(test_df['climateregions__climateregion'])
train_df['climateregions__climateregion']

0         0
1         0
2         0
3         0
4         0
         ..
375729    9
375730    9
375731    9
375732    9
375733    9
Name: climateregions__climateregion, Length: 375734, dtype: int64

In [12]:
# Replace latitude and lontitude info with 514 unique locations
train_df['unique_loc'] = train_df.groupby(['lat','lon']).ngroup()
test_df['unique_loc'] = test_df.groupby(['lat','lon']).ngroup()

In [13]:
# Replace start date with year, month and day
train_df['year'] = train_df['startdate'].dt.year
train_df['month'] = train_df['startdate'].dt.month
train_df['day_of_year'] = train_df['startdate'].dt.dayofyear

test_df['year'] = test_df['startdate'].dt.year
test_df['month'] = test_df['startdate'].dt.month
test_df['day_of_year'] = test_df['startdate'].dt.dayofyear

In [14]:
# Drop uncessary columns including the target column
drop_cols = ['index', 'startdate','lat', 'lon']

target = train_df['contest-tmp2m-14d__tmp2m']
train_df = train_df.drop(columns=drop_cols)
train_df = train_df.drop(columns=['contest-tmp2m-14d__tmp2m'])
test_df = test_df.drop(columns=drop_cols)

In [15]:
corr_test = test_df.corr().abs()

In [16]:
upper_tri_test = corr_test.where(np.triu(np.ones(corr_test.shape),k=1).astype(np.bool))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [17]:
to_drop_test = [column for column in upper_tri_test.columns if any(upper_tri_test[column] > 0.9)]

In [18]:
train_df = train_df.drop(columns=to_drop_test)
test_df = test_df.drop(columns=to_drop_test)

In [19]:
%%capture
!pip install --pre pycaret

In [20]:
from pycaret.regression import *
s = setup(train_df, target = target,
         session_id = 123,
         normalize = True,
         normalize_method = 'minmax',
         use_gpu = True,
         #log_experiment = 'wandb',
         #experiment_name = 'climate_train',
         #profile = True,
         train_size = 0.8,
         fold_strategy = 'timeseries',
         #numeric_features = ['climateregions__climateregion','month','unique_loc']
         remove_multicollinearity = False,
         pca = False,
         )

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,123
1,Target,contest-tmp2m-14d__tmp2m
2,Target type,Regression
3,Original data shape,"(375734, 125)"
4,Transformed data shape,"(375734, 125)"
5,Transformed train set shape,"(300587, 125)"
6,Transformed test set shape,"(75147, 125)"
7,Numeric features,124
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


In [21]:
models()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [22]:
compare_models(exclude=['gbr','rf','et'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9032,1.3434,1.1590,0.9862,0.1886,0.6008,11.8870
dt,Decision Tree Regressor,0.8282,1.7808,1.2904,0.9818,0.2009,0.5995,24.3930
knn,K Neighbors Regressor,1.0100,1.9678,1.3858,0.9798,0.2073,0.6004,128.4880
lr,Linear Regression,1.3749,3.1586,1.7772,0.9676,0.2568,0.7920,5.5240
ridge,Ridge Regression,1.3750,3.1595,1.7775,0.9676,0.2570,0.7932,1.4410
br,Bayesian Ridge,1.3749,3.1586,1.7772,0.9676,0.2568,0.7922,7.0180
huber,Huber Regressor,1.3987,3.3401,1.8276,0.9657,0.2653,0.7951,42.3380
omp,Orthogonal Matching Pursuit,1.5206,3.8896,1.9722,0.9601,0.2825,0.9083,1.5940
ada,AdaBoost Regressor,2.0112,6.2144,2.4927,0.9362,0.3422,1.4728,193.3820
par,Passive Aggressive Regressor,2.0885,6.7468,2.5689,0.9308,0.3628,1.3556,2.9980


Processing:   0%|          | 0/73 [00:00<?, ?it/s]

LGBMRegressor(random_state=123)

## LightGBM

In [23]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9235,1.4240,1.1933,0.9855,0.1913,0.3947
1,0.9148,1.3805,1.1750,0.9860,0.1897,0.6409
2,0.9030,1.3493,1.1616,0.9863,0.1903,0.3480
3,0.8971,1.3243,1.1508,0.9862,0.1878,0.6198
4,0.8959,1.3109,1.1450,0.9866,0.1888,0.4436
5,0.9082,1.3565,1.1647,0.9860,0.1890,0.3664
6,0.8980,1.3243,1.1508,0.9864,0.1865,1.0461
7,0.8919,1.3117,1.1453,0.9865,0.1861,0.5679
8,0.9010,1.3292,1.1529,0.9862,0.1897,0.7867


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tune = tune_model(lightgbm,search_library = 'optuna')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:35:52
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2023-03-13 10:35:53,428] Searching the best hyperparameters using 300587 samples...


[LightGBM] [Warning] bagging_fraction is set=0.6731772703817784, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6731772703817784
[LightGBM] [Warning] feature_fraction is set=0.790149835537747, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.790149835537747
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.574303301474728, subsample=1.0 will be ignored. Current value: bagging_fraction=0.574303301474728
[LightGBM] [Warning] feature_fraction is set=0.5724507249108499, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5724507249108499
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=0.7764508882444572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7764508882444572
[LightGBM] [Warning] feature_fraction i

## XGBoost

In [ ]:
xgboost = create_model('xgboost')

In [ ]:
tune_xgb = tune_model(xgboost,search_library = 'optuna')

## Catboost

In [ ]:
catboost = create_model('catboost')

In [ ]:
tune_cat = tune_model(catboost,search_library = 'optuna')

In [ ]:
unseen_predictions_gbm = predict_model(tune_cat, data=test_df)
unseen_predictions_gbm

In [ ]:
unseen_predictions_gbm.to_csv('sub_1-107.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(unseen_predictions_gbm['prediction_label'], bins=50)
plt.show()